# How to use xarray's `apply_ufunc` method

Vectorisation is key in improving the performance of array manipulation.
So much so that between `numpy`, `xarray`, and `scipy` practially all conceivable operations have already been vectorised.

But as we learn from time to time, researchers always find new an exciting calculations to do, and out come the for loops.

Let's first load the relevant modules, then look at a few examples.

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from dask.distributed import Client
import dask.array as da
import scipy.stats as stats
import warnings

In [2]:
# Initialise dask if not there already
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    try:
        c
    except NameError as ne:
        c = Client(n_workers=4, threads_per_worker=1, memory_limit='3.5GB')

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0

In [3]:
c

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 13.04 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:63498,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 13.04 GiB
Comm: tcp://127.0.0.1:63520,Total threads: 1
Dashboard: http://127.0.0.1:63525/status,Memory: 3.26 GiB
Nanny: tcp://127.0.0.1:63501,


## Generate some data for the testing

The following function just creates some data array with random numbers in it.

In [4]:
def create_dataarray(nlat, nlon, ntime=None, seed=1000):

    np.random.seed(seed)
    
    lat = np.linspace(-90, 90, nlat, endpoint=True)
    lat = xr.DataArray(lat, dims=('lat',), coords={'lat': lat}, attrs={'units': 'degree_north', 'name': 'Latitude'})
    
    lon = np.linspace(-180, 180, nlon+1, endpoint=True)[1:]
    lon = xr.DataArray(lon, dims=('lon',), coords={'lon': lon}, attrs={'units': 'degree_east', 'name': 'Longitude'})

    if ntime is not None:
        time = pd.date_range(start='2000-01-01', freq='D', periods=ntime)
        time = xr.DataArray(time, dims=('time',))
        return xr.DataArray(
            np.random.random([ntime, nlat, nlon]), 
            dims=('time', 'lat', 'lon'),
            coords={'time': time, 'lat': lat, 'lon': lon},
            attrs={'name': 'random'}
        )

    return xr.DataArray(
        np.random.random([nlat, nlon]), 
        dims=('lat', 'lon'),
        coords={'lat': lat, 'lon': lon},
        attrs={'name': 'random'}
    )

## Scalar Functions
First, let's look at a function that takes one value and outputs another. Because I'm lazy, I'll use a function that maxes the input value out at a certain value

In [5]:
def limit(value, limit=0.5):
    return limit if limit < value else value

print(f"{limit(0.2)=}")
print(f"{limit(1.2)=}")
print(f"{limit(0.2, 0.1)=}")

limit(0.2)=0.2
limit(1.2)=0.5
limit(0.2, 0.1)=0.1


First, we create a very small dataset, so that we can see exactly what's going on.

In [6]:
small_da = create_dataarray(1, 5)
small_da

<xarray.DataArray (lat: 1, lon: 5)>
array([[0.65358959, 0.11500694, 0.95028286, 0.4821914 , 0.87247454]])
Coordinates:
  * lat      (lat) float64 -90.0
  * lon      (lon) float64 -108.0 -36.0 36.0 108.0 180.0
Attributes:
    name:     random

Our function expects scalar values, so we need to tell the call to `apply_ufunc` that it needs to vectorise it.

In [7]:
xr.apply_ufunc(limit, small_da, vectorize=True)

<xarray.DataArray (lat: 1, lon: 5)>
array([[0.5       , 0.11500694, 0.5       , 0.4821914 , 0.5       ]])
Coordinates:
  * lat      (lat) float64 -90.0
  * lon      (lon) float64 -108.0 -36.0 36.0 108.0 180.0

This worked just as expected: All the values that used to be larger than 0.5 are now capped at that value

We can supply additional parameters to the function by using the `kwargs` parameter:

In [8]:
xr.apply_ufunc(limit, small_da, vectorize=True, kwargs={'limit': 0.3})

<xarray.DataArray (lat: 1, lon: 5)>
array([[0.3       , 0.11500694, 0.3       , 0.3       , 0.3       ]])
Coordinates:
  * lat      (lat) float64 -90.0
  * lon      (lon) float64 -108.0 -36.0 36.0 108.0 180.0

Now the cutoff has been lowered to 0.3, and more values have been capped.

With `vectorize=True`, the `apply_ufunc` method under the hood uses the `numpy.vectorize` method to vectorise the input. 
Doing this explicitly would look something like this:

In [9]:
xr.apply_ufunc(np.vectorize(limit), small_da)

<xarray.DataArray (lat: 1, lon: 5)>
array([[0.5       , 0.11500694, 0.5       , 0.4821914 , 0.5       ]])
Coordinates:
  * lat      (lat) float64 -90.0
  * lon      (lon) float64 -108.0 -36.0 36.0 108.0 180.0

The [apply_ufunc](https://docs.xarray.dev/en/stable/generated/xarray.apply_ufunc.html) documentation mentiones that this might be suboptimal performance wise, and that pre-vectorised functions should be used.

A vectorised function already expects an array. 
In our case it would look something like this:

In [10]:
def limit_v(array, value=0.5):
    return np.where(array>value, value, array)

xr.apply_ufunc(limit_v, small_da)

<xarray.DataArray (lat: 1, lon: 5)>
array([[0.5       , 0.11500694, 0.5       , 0.4821914 , 0.5       ]])
Coordinates:
  * lat      (lat) float64 -90.0
  * lon      (lon) float64 -108.0 -36.0 36.0 108.0 180.0

### Performance Comparison of Vectorisation

In [11]:
large_da = create_dataarray(180, 360)

In [12]:
%%time
limited = large_da.copy()
for y in range(len(large_da.lat)):
    for x in range(len(large_da.lon)):
        limited[y, x] = limit(large_da[y, x])

CPU times: user 22.2 s, sys: 145 ms, total: 22.4 s
Wall time: 22.4 s


In [13]:
%%timeit
limited = xr.apply_ufunc(limit, large_da, vectorize=True)

18.2 ms ± 51.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
%%timeit
limited = xr.apply_ufunc(limit_v, large_da)

127 µs ± 940 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Conclusion for scalar functions
It is pretty clear that we gain several orders of magnitude performance boost over the explicit loops even with the automatically vectorised call. 

But the best is if we have a function that is already vectorised, so works on arrays.
Of course this is then no longer a scalar function.

## Reduction functions
Next we want to see what we need to do when our function reduces an array dimension to a single value.

For example, this returns the index of the largest value along an axis:

In [15]:
def argmax(array):
    return array.argmax()

For this example, we create a small data array with a time dimension, and we want to get the index of the largest value along that time dimension

In [16]:
small_da = create_dataarray(1, 2, 5)
small_da

<xarray.DataArray (time: 5, lat: 1, lon: 2)>
array([[[0.65358959, 0.11500694]],

       [[0.95028286, 0.4821914 ]],

       [[0.87247454, 0.21233268]],

       [[0.04070962, 0.39719446]],

       [[0.2331322 , 0.84174072]]])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 ... 2000-01-05
  * lat      (lat) float64 -90.0
  * lon      (lon) float64 0.0 180.0
Attributes:
    name:     random

### Core dimensions
The concept of core dimensions isn't very well explained in the documentation. 

Our function wants to look along the `time` dimension for the largest value. 
In the speak of `xarray`, this makes `time` the core dimension for the input of the function

`input_core_dims` is a **list** of **tuples** of dimension names.
Because the function has only a single input array, the outer list of the `input_core_dims` has only one element, the tuple `('time',)`.
Because `time` is the only core dimension of the first array, the first (and only) tuple has only this name as its element.

In [17]:
xr.apply_ufunc(argmax, small_da, input_core_dims=[('time',)], vectorize=True)

<xarray.DataArray (lat: 1, lon: 2)>
array([[1, 4]])
Coordinates:
  * lat      (lat) float64 -90.0
  * lon      (lon) float64 0.0 180.0

### Vectorisation, again
Again, we want to see whether we can manually vectorise a function. 
But this time we don't use the `np.vectorize` method, instead we write the function in such a way that it is vectorised from the get-go.

With `vectorize=True` the data is spliced into 1-D arrays and individually passed to the function. 

With `vectorize=False` (or no `vectorize` at all), the full array is passed to the function in one go, except that the core dimension is placed at the very end. 

We can use this:

In [18]:
def argmax_v(array):
    return array.argmax(axis=-1)

In [19]:
large_da = create_dataarray(181, 360, 365)

In [20]:
%%timeit
am = xr.apply_ufunc(argmax, large_da, input_core_dims=[('time',)], vectorize=True)

197 ms ± 5.35 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
%%timeit
am_v = xr.apply_ufunc(argmax_v, large_da, input_core_dims=[('time',)])

42.3 ms ± 284 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


We can see that this time also there is a noticeable improvement of performance, the vectorised version ran twice as fast.

## Functions that take and return an array

In this case, we want to invert the values along one dimension.

In [22]:
def invert(array):
    return array[ ..., -1::-1]

print(invert(np.arange(4)))

[3 2 1 0]


```{admonition} What is ...
numpy has the notation `...` to note that there may be more dimensions than we're using. 
The syntax `array[ ..., -1::-1 ]` means, depending on the number of dimensions of `array`:
`array[-1::-1]` for a 1-d array, `array[:, -1::-1]` for a 2-d array, `array[:, :, -1::-1]` for a 3-d array and so forth.
```


In [23]:
small_da = create_dataarray(2, 5)
small_da

<xarray.DataArray (lat: 2, lon: 5)>
array([[0.65358959, 0.11500694, 0.95028286, 0.4821914 , 0.87247454],
       [0.21233268, 0.04070962, 0.39719446, 0.2331322 , 0.84174072]])
Coordinates:
  * lat      (lat) float64 -90.0 90.0
  * lon      (lon) float64 -108.0 -36.0 36.0 108.0 180.0
Attributes:
    name:     random

In [24]:
xr.apply_ufunc(
    invert, small_da, input_core_dims=[('lon',)], 
    output_core_dims=[('lon',)]
)

<xarray.DataArray (lat: 2, lon: 5)>
array([[0.87247454, 0.4821914 , 0.95028286, 0.11500694, 0.65358959],
       [0.84174072, 0.2331322 , 0.39719446, 0.04070962, 0.21233268]])
Coordinates:
  * lat      (lat) float64 -90.0 90.0
  * lon      (lon) float64 -108.0 -36.0 36.0 108.0 180.0

Because the input core dimension and the output core dimension have the same name, the `apply_ufunc` method assumes that it uses the same coordinates. 

The fact that our function has inverted the array along the longitude axis is not known to the `apply_ufunc` method, and so the longitude values are still in the original, ascending order.

### Differing length between input and output core dimension

The input and output of a function might be arrays of different length.
In this case we're simply capping the time dimension to 3 values

In [25]:
small_da = create_dataarray(1, 2, 5)
small_da

<xarray.DataArray (time: 5, lat: 1, lon: 2)>
array([[[0.65358959, 0.11500694]],

       [[0.95028286, 0.4821914 ]],

       [[0.87247454, 0.21233268]],

       [[0.04070962, 0.39719446]],

       [[0.2331322 , 0.84174072]]])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 ... 2000-01-05
  * lat      (lat) float64 -90.0
  * lon      (lon) float64 0.0 180.0
Attributes:
    name:     random

In [26]:
def first_three(array):
    return array[ ..., :3]

If I try as above to give it the same name, I will get an error. 
The same name suggests that it should have the same coordinates, but the length of the dimension no longer matches.

I have two options:
1. Give the output core dimension a new name
2. Tell it that it should ignore the change.
   This is done by using the `exclude_dim` parameter.

In [27]:
xr.apply_ufunc(first_three, small_da, input_core_dims=[('time',)], output_core_dims=[('new_time',)])

<xarray.DataArray (lat: 1, lon: 2, new_time: 3)>
array([[[0.65358959, 0.95028286, 0.87247454],
        [0.11500694, 0.4821914 , 0.21233268]]])
Coordinates:
  * lat      (lat) float64 -90.0
  * lon      (lon) float64 0.0 180.0
Dimensions without coordinates: new_time

In [28]:
xr.apply_ufunc(
    first_three, small_da, 
    input_core_dims=[('time',)], output_core_dims=[('time',)],
    exclude_dims=set(['time'])
)

<xarray.DataArray (lat: 1, lon: 2, time: 3)>
array([[[0.65358959, 0.95028286, 0.87247454],
        [0.11500694, 0.4821914 , 0.21233268]]])
Coordinates:
  * lat      (lat) float64 -90.0
  * lon      (lon) float64 0.0 180.0
Dimensions without coordinates: time

## Applying a ufunc over multiple arrays

Now that we have a rough understaning of how the `xarray.apply_ufunc` works, let's try something more complex.
We want to calculate the Pearson Correlation Coefficient between the input dataset and another array.

In [29]:
small_da = create_dataarray(1, 2, 5)
comparison_da = small_da.isel(lat=0, lon=-1) # This way exactly one will have perfect correlation

I'm using a slightly modified version straight out of the `numpy.vectorize` [API reference](https://numpy.org/doc/stable/reference/generated/numpy.vectorize.html).

The `signature` parameter describes how the function expects and returns values. 
In this case it expects two 1-d arrays with the same lenth, and returns a single scalar.

In [30]:
def my_pearson(x, y):
    return stats.pearsonr(x, y).statistic

my_pearson_v = np.vectorize(
    my_pearson,
    signature='(n), (n) -> ()'
)

The two different arrays are passed after each other.
Note that the `input_core_dims` now needs to reference both core dimensions of the two input arrays.

In [31]:
xr.apply_ufunc(
    my_pearson_v, 
    small_da, 
    comparison_da,
    input_core_dims=[('time',), ('time',)]
)

<xarray.DataArray (lat: 1, lon: 2)>
array([[-0.4356561,  1.       ]])
Coordinates:
  * lat      (lat) float64 -90.0
  * lon      (lon) float64 0.0 180.0

We see indeed perfect correlation for the last longitude.

## Using dask for large datasets

Normally we don't run our correlation experiment on such small and easy-to-handle datasets.
For larger datasets, we use `dask` for parallelisation.

Let's create a slightly larger dataset.

In [32]:
large_da = create_dataarray(181, 360, 3650)
comparison_da = large_da.isel(lat=0, lon=-1)

Let's first try this without dask to see some baseline

In [33]:
%%time
o = xr.apply_ufunc(
    my_pearson_v,
    large_da,
    comparison_da,
    input_core_dims=[('time',), ('time',)],
)

CPU times: user 38.6 s, sys: 561 ms, total: 39.1 s
Wall time: 38.9 s


By rechunking, we turn the `large_da` data array into a dask array.
Let's check how long it takes to compute the correlation without parallelisation.

In [34]:
large_da = large_da.chunk({'lat': 91, 'lon': 90})
large_da

<xarray.DataArray (time: 3650, lat: 181, lon: 360)>
dask.array<xarray-<this-array>, shape=(3650, 181, 360), dtype=float64, chunksize=(3650, 91, 90), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 ... 2009-12-28
  * lat      (lat) float64 -90.0 -89.0 -88.0 -87.0 -86.0 ... 87.0 88.0 89.0 90.0
  * lon      (lon) float64 -179.0 -178.0 -177.0 -176.0 ... 178.0 179.0 180.0
Attributes:
    name:     random

For dask, we need two new parameters:
- `dask="parallelized"` to tell the system to actually use dask, and
- `output_dtypes=["float"]` to tell it what to expect as a result.

In [35]:
r = xr.apply_ufunc(
    my_pearson_v,
    large_da,
    comparison_da,
    input_core_dims=[('time',), ('time',)],
    dask='parallelized',
    output_dtypes=['float']
)

Because of lazy loading, the cell above returned immediately. 
It hasn't done any of the calculations yet, just stored what it's supposed to to.

Only in the next cell, where we tell it to actually compute the output, will it take time.

In [36]:
%%time
p = r.compute()
p

CPU times: user 976 ms, sys: 922 ms, total: 1.9 s
Wall time: 12.3 s


<xarray.DataArray (lat: 181, lon: 360)>
array([[ 7.49156667e-03, -1.05611671e-02, -6.57951424e-03, ...,
        -2.13526211e-03, -6.70029486e-03,  1.00000000e+00],
       [ 1.45195946e-02,  3.92027344e-03,  1.36661369e-02, ...,
        -8.23021313e-03,  5.88698092e-03,  9.57717761e-03],
       [ 9.63523125e-03,  5.45873825e-04,  2.91139380e-03, ...,
         5.17681565e-03,  6.33934265e-03, -2.11432513e-03],
       ...,
       [ 2.99324695e-02,  2.14475632e-02, -2.38409887e-02, ...,
        -8.38419120e-04, -1.74913441e-02, -3.59145362e-02],
       [ 5.00181784e-03, -5.90834912e-03,  1.15881635e-02, ...,
         1.84094023e-02,  7.76888293e-03, -2.42987961e-02],
       [-3.11181858e-02,  3.26837453e-02, -1.08989192e-02, ...,
        -1.16345114e-02,  2.47818545e-02, -9.01637243e-03]])
Coordinates:
  * lat      (lat) float64 -90.0 -89.0 -88.0 -87.0 -86.0 ... 87.0 88.0 89.0 90.0
  * lon      (lon) float64 -179.0 -178.0 -177.0 -176.0 ... 178.0 179.0 180.0

In [37]:
# Ensure that the perfect correlation still exists 
# where we have extracted the actual comparison array
p.isel(lat=0, lon=-1).item()

0.9999999999999998

## Dask's Generalised Universal Functions

`dask.array` has its own method `gufunc` similar to `numpy.vectorize` to create generalised universal functions that create functions optimised for dask parallelisation.

It is described in detail [here](https://docs.dask.org/en/latest/array-gufunc.html)

In [38]:
my_pearson_g = da.gufunc(
    my_pearson, 
    signature='(n),(n) -> ()',
    vectorize=True,
    output_dtypes=np.float64
)

In [39]:
r = xr.apply_ufunc(
    my_pearson_g,
    large_da,
    comparison_da,
    input_core_dims=[('time',), ('time',)],
    dask='allowed',
)

In [40]:
%%time
p2 = r.compute()
p2.isel(lat=0, lon=-1).item()

CPU times: user 824 ms, sys: 930 ms, total: 1.75 s
Wall time: 11.4 s


0.9999999999999998

## Conclusion

Xarray's `apply_ufunc` method can seriously speed up calculations along dimensions in multi-dimensional xarray dataarrays.

This option is very powerful, but as always this comes with added complexity.

I hope this short document will help you get in the mindset of the function and help you master your universal functions.